# Deep learning
### Back propagation

In [8]:
from tensorflow import keras
from keras.datasets import mnist
import time
import numpy as np
import idx2numpy
import pandas as pd

IMG_SIZE = 28
INNER_LAYER_SIZE = 300
OUT_SIZE = 10

(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [9]:
def dataConverter (x, y, imgSIZE = 28, outSIZE = 10):
    y = np.eye(outSIZE)[y]
    x = x/255
    x = x.reshape((x.shape[0], imgSIZE * imgSIZE))
    return x,y

def printData (x,y):
    print('X_train: ' + str(x.shape))
    print('Y_train: ' + str(y.shape))
    
train_X, train_y = dataConverter(train_X, train_y)
test_X, test_y = dataConverter(test_X, test_y)

printData(train_X, train_y)
printData(test_X,test_y)

X_train: (60000, 784)
Y_train: (60000, 10)
X_train: (10000, 784)
Y_train: (10000, 10)


In [10]:
def ReLU(x):
    return np.maximum(0, x)

def ReLU_derivative(values):
    result = [1 if x > 0 else 0 for x in values]
    return result

def softmax(x):
    exp = np.exp(x)
    return exp / np.sum(exp, axis = 1, keepdims = True)

def crossEntropyLoss(x, y):
    return np.mean(-np.sum(x * np.log(y), axis=1))

def precision(x1, x2):
    return np.mean(np.argmax(x1, axis=1) == np.argmax(x2, axis=1))

In [11]:
class NeuralNetwork():
    
    def __init__(self, nnDims = [IMG_SIZE*IMG_SIZE, INNER_LAYER_SIZE, OUT_SIZE], learningRate=0.1, batchSize = 32 , epochs = 20):
        inpL = nnDims[0]
        hidL = nnDims[1]
        outL = nnDims[2]                
        self.learningRate = learningRate
        self.batchSize = batchSize
        self.epochs = epochs
      
        self.w1 = np.random.normal(0, np.sqrt(2 / inpL), [inpL, hidL])
        self.b1 = np.zeros((1, hidL))
        self.w2 = np.random.normal(0, np.sqrt(2 / (inpL + outL)), [hidL, outL])
        self.b2 = np.zeros((1, outL))
        
    def forward(self, x):
        self.v1 = x @ self.w1 + self.b1
        self.u1 = ReLU(self.v1)
        self.v2 = self.u1 @ self.w2 + self.b2
        self.u2 = softmax(self.v2)

    def backPropagation(self, x_train, y_train):
        dv2 = (self.u2 - y_train) / self.u2.shape[0]
        dw1 = self.u1.T @ dv2
        db1 = np.sum(dv2, axis = 0, keepdims = True)        
        dv1 = dv2 @ self.w2.T * np.where (self.v1 > 0, 1, 0)
        dw0 = x_train.T @ dv1
        db0 = np.sum(dv1, axis = 0, keepdims = True)
        
        self.w2 -= self.learningRate * dw1
        self.b2 -= self.learningRate * db1
        self.w1 -= self.learningRate * dw0        
        self.b1 -= self.learningRate * db0
        
    def train(self, x_train, y_train):
        epochTime = []
        epochLoss = []
        epochPrecision = []
        fullTime = time.time()
        for epoch in range(self.epochs):
            startEpoch = time.time()
            startPos = 0
            while startPos + self.batchSize <= len(x_train):
                x_cut = x_train[startPos : startPos + self.batchSize]
                y_cut = y_train[startPos : startPos + self.batchSize]
                self.forward(x_cut)
                self.backPropagation(x_cut, y_cut)
                startPos += self.batchSize           
            self.forward(x_train)
            epochTime.append(time.time() - startEpoch)
            epochLoss.append(crossEntropyLoss(y_train, self.u2))
            epochPrecision.append(precision(y_train, self.u2))
            
        fullTime = (time.time() - fullTime)
        df = pd.DataFrame({'Epoch iteration': [*range(1, self.epochs + 1, 1)],
                          'Time spent': epochTime,
                          'Cross-entropy val': epochLoss,
                          'Precision': epochPrecision})
        df.set_index('Epoch iteration', inplace=True)
        pd.set_option('display.width', 100)
        print(df)
        print('Overall training time', fullTime)

    def test(self, x_test, y_test):
        self.forward(x_test)
        print('Test Cross-entropy Value', crossEntropyLoss(y_test, self.u2))
        print('Test Precision', precision(y_test, self.u2))

In [12]:
nn = NeuralNetwork()
nn.train(train_X, train_y)

                 Time spent  Cross-entropy val  Precision
Epoch iteration                                          
1                  3.099235           0.180703   0.946267
2                  3.092032           0.117660   0.965050
3                  3.065874           0.087943   0.973950
4                  3.114700           0.068477   0.979833
5                  3.254263           0.055585   0.983883
6                  3.282206           0.046757   0.986217
7                  3.077783           0.039650   0.988533
8                  3.349922           0.033985   0.990567
9                  3.327118           0.029160   0.992117
10                 3.175586           0.025297   0.993383
11                 3.202433           0.021961   0.994583
12                 3.106688           0.019192   0.995600
13                 3.128008           0.017119   0.996117
14                 3.186492           0.015361   0.996583
15                 3.239408           0.013858   0.997000
16            

In [13]:
nn.test(test_X, test_y)

Test Cross-entropy Value 0.06838752439217122
Test Precision 0.9796
